In [1]:
from pyspark import SparkConf, SparkContext
sc = SparkContext(conf=SparkConf().setAppName("MyApp").setMaster("yarn"))

In [2]:
import re

def parse_article(line):
    try:
        article_id, text = line.rstrip().split('\t', 1)
        text = re.sub("^\W+|\W+$", "", text, flags=re.UNICODE)
        words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
        return words
    except ValueError as e:
        return []

wiki = sc.textFile("/data/wiki/en_articles_part/articles-part", 4).map(parse_article)

In [3]:
pairs = []

words = wiki.collect()
for i in range(len(words)):
    partition = [x.lower() for x in words[i]]
    indexes = [i for i, x in enumerate(partition) if x.startswith('narodnaya')]
    for ind in indexes:
        pairs.append((partition[ind].strip(), partition[ind+1].strip()))

In [4]:
for i in range(len(pairs)):
    pairs[i] = '_'.join(str(x) for x in pairs[i])

In [5]:
from collections import Counter

pairs = dict(Counter(pairs))

In [6]:
for k in sorted(Counter(pairs).keys()):
    print(k + '\t' + str(pairs[k]))

narodnaya_gazeta	1
narodnaya_volya	9
